In [1]:
import os
import sys

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob
for f in glob.glob("./*.pth"):
    print(f)

.\efficientnet-b4_best_fold0.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0_epoch120.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0_epoch30.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0_epoch60.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0_epoch90.pth
.\efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_model_fold0.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_best_fold0.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_best_fold0_epoch120.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_best_fold0_epoch30.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_best_fold0_epoch60.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_best_fold0_epoch90.pth
.\resnet-101_250_freeze_gridmask_augmix_fulltrain_model_fold0.pth
.\resnet-18_best_fold0.pth
.\resnet-18_best_fold0_epoch120.pth
.\resnet-18_best_fold0_epo

In [4]:
import pandas as pd
import cv2
import numpy as np
import torch
import albumentations
import matplotlib.pyplot as plt
import glob
import math
from PIL import Image as Image
import torchvision
from torchvision import transforms
import sklearn.metrics
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.nn.functional as FT
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm as tqdm
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.augmentations import functional as F
import albumentations as A
device = torch.device("cuda")
import time
import torchvision.datasets as dset

In [5]:
kernel_type = "efficientnet-b7_600_freeze_gridmask_augmix_fulltrain"
RANDOM_STATE = 47
MEAN = [0.485, 0.456, 0.406] 
STD = [0.229, 0.224, 0.225]
n_folds = 5
n_epochs = 120
HEIGHT = 224
WIDTH = 224
num_workers = 0
batch_size = 32
data_dir = "./Dataset/"

idx2class = {i:class_name for i,class_name in enumerate(["angry", "happy", "sad", "surprised", "Unknown"]) }
class2idx = {class_name:i for i,class_name in enumerate(["angry", "happy", "sad", "surprised", "Unknown"]) }

out_dim = len(["angry", "happy", "sad", "surprised", "Unknown"])
fold = 0
out_dim  = len(idx2class)
init_lr = 0.01

In [6]:
model_file = f"./{kernel_type}_best_fold0.pth"

In [7]:
class Albumentations_cls():
    def __init__(self, augmentations):
        self.augmentations  = A.Compose(augmentations)
    
    def __call__(self, image):
        return self.augmentations(image = image)["image"]

In [8]:
class DatasetTrain(torch.utils.data.Dataset):
    
    def __init__(self, dataset, transform=None):
        super(DatasetTrain, self).__init__()
        np.random.seed(0)
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return  len(self.dataset.imgs)

    def __getitem__(self, index):
        image1_name = self.dataset.imgs[index]
        
        image1 = cv2.imread(image1_name[0])

        if self.transform:
            image1 = self.transform(image1)
            image1 = transforms.Normalize(MEAN, STD)(image1)
        
        return image1, image1_name[0].split("\\")[-1]

In [9]:
preprocess = [
    A.Resize(height = HEIGHT, width = WIDTH, always_apply=True),
]



transforms_val = transforms.Compose([
    np.uint8,
    # transforms.Normalize(mean = MEAN, std = STD),
    Albumentations_cls(preprocess),
    transforms.ToTensor(),
    
])
transforms_orig = transforms.Compose([
    np.uint8,
    Albumentations_cls(preprocess),
    transforms.ToTensor(),   
])

In [27]:
pretrained_dict = {f'efficientnet-b{i}': path for i,path in enumerate(sorted(glob.glob('../input/efficientnet-pytorch/*pth')))}
from efficientnet_pytorch import EfficientNet
import torchvision.models as models
sigmoid = nn.Sigmoid()
class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i + i*(1 - sigmoid_i))
swish = Swish.apply
class Swish_module(nn.Module):
    def forward(self, x):
        return swish(x)
    
swish_layer = Swish_module()

def relu_fn(x):
    return swish_layer(x)

class GlobalAvgPool(nn.Module):
        def __init__(self):
            super(GlobalAvgPool, self).__init__()
        def forward(self, x):
            return x.view(*(x.shape[:-2]),-1).mean(-1)


class Seq_Ex_Block(nn.Module):
        def __init__(self, in_ch, r):
            super(Seq_Ex_Block, self).__init__()
            self.se = nn.Sequential(
                GlobalAvgPool(),
                nn.Linear(in_ch, in_ch//r),
                nn.ReLU(inplace=True),
                nn.Linear(in_ch//r, in_ch),
                nn.Sigmoid()
            )

        def forward(self, x):
            se_weight = self.se(x).unsqueeze(-1).unsqueeze(-1)
            #print(f'x:{x.sum()}, x_se:{x.mul(se_weight).sum()}')
            return x.mul(se_weight)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
      
class ClassifierNew(nn.Module):
    def __init__(self, inp = 2208, h1=1024, out = 102, d=0.35):
        super().__init__()
        self.ap = nn.AdaptiveAvgPool2d((1,1))
        self.mp = nn.AdaptiveMaxPool2d((1,1))
        self.fla = Flatten()
        self.bn0 = nn.BatchNorm1d(inp*2,eps=1e-05, momentum=0.1, affine=True)
        self.dropout0 = nn.Dropout(d)
        self.fc1 = nn.Linear(inp*2, h1)
        self.bn1 = nn.BatchNorm1d(h1,eps=1e-05, momentum=0.1, affine=True)
        self.dropout1 = nn.Dropout(d)
        self.fc2 = nn.Linear(h1, out)
        self.activation = nn.Softmax()
        
    def forward(self, x):
        ap = self.ap(x)
        mp = self.mp(x)
        x = torch.cat((ap,mp),dim=1)
        x = self.fla(x)
        x = self.bn0(x)
        x = self.dropout0(x)
        x = FT.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout1(x)         
        x = self.fc2(x)
        x = self.activation(x)
        return x
class NeuralNet(nn.Module):
    def __init__(self, pretrained = True, Freeze_base = False, layers_freeze = None):
        super(NeuralNet, self).__init__()
        self.cnn = models.resnet101(pretrained= pretrained)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-2])
        if Freeze_base:
            if layers_freeze == None:
                for p in self.cnn.parameters():
                    p.requires_grad = False
            else:
                c = 0
                for p in self.cnn.parameters():
                    c+=1
                    if c < layers_freeze:
                        p.requires_grad = False
                    else:
                        p.requires_grad = True
                        

        self.fc = ClassifierNew(2048, 1024, 4, 0.35)
    def forward(self, input):
        x = self.cnn(input)
        x = self.fc(x)
        return x
class EfficientNet_NeuralNet(nn.Module):
    def __init__(self, pretrained = True, Freeze_base = False, layers_freeze = None):
        super(EfficientNet_NeuralNet, self).__init__()
        
        self.cnn = EfficientNet.from_pretrained('efficientnet-b7')
        self.cnn._avg_pooling = nn.Identity()
        self.cnn._dropout = nn.Identity()
        self.cnn._swish = nn.Identity()
        if Freeze_base:
            if layers_freeze == None:
                for p in self.cnn.parameters():
                    p.requires_grad = False
            else:
                c = 0
                for p in self.cnn.parameters():
                    c+=1
                    if c < layers_freeze:
                        p.requires_grad = False
                    else:
                        p.requires_grad = True
        self.fc = ClassifierNew(2560, 1024, 4, 0.35)
        self.cnn._fc = nn.Identity()
    def forward(self, input):
        x = self.cnn.extract_features(input)
        x = self.fc(x)
        return x

In [28]:
def test_epoch(test_dataloader ):
    model.eval()
    Logits = []
    outputs = []
    test_labels = []
    with torch.no_grad():
        for(data, label) in tqdm(test_dataloader):
            data = data.to(device)
            logits = model(data)
            
            for i in label:
                test_labels.append(i)
                df["Frame_ID"].append(i)
            preds = logits.argmax(1).detach()
            for p in preds:
                df["Emotion"].append(p.item())
            outputs.append(preds)
    return outputs, Logits, test_labels

In [29]:
df = {
    "Frame_ID":[],
    "Emotion":[]
}

In [30]:
test_dataset = dset.ImageFolder(root = "./Dataset/test")
td = DatasetTrain(test_dataset, transforms_val)
tdl = torch.utils.data.DataLoader(td, batch_size=batch_size, shuffle=False,num_workers=num_workers)

In [31]:
# model = NeuralNet(pretrained= True, Freeze_base= True, layers_freeze = 160)
model = EfficientNet_NeuralNet(pretrained= True, Freeze_base= True, layers_freeze = 160)
model = model.to(device)
model.load_state_dict(torch.load(model_file))

Loaded pretrained weights for efficientnet-b7


<All keys matched successfully>

In [32]:
outputs, logits, test_labels = test_epoch(tdl)

100%|██████████| 5/5 [00:04<00:00,  1.23it/s]


In [33]:
df = pd.DataFrame(df)

In [34]:
df.Emotion = df.Emotion.map(idx2class)

In [35]:
df.Emotion.value_counts()

angry        44
surprised    41
happy        32
sad          26
Name: Emotion, dtype: int64

In [36]:
df_test = pd.read_csv(f"{data_dir}/Test.csv")
df_test = df_test.merge(df, "left", on = "Frame_ID", )

In [37]:
df_test = df_test.fillna("Unknown")

In [38]:
df_test.Emotion.value_counts()

angry        44
Unknown      43
surprised    41
happy        32
sad          26
Name: Emotion, dtype: int64

In [39]:
df_test.shape

(186, 2)

In [40]:
save = model_file.split("/")[-1].split(".")[0]

In [41]:
df_test.to_csv(f"./submissions/{save}_submission.csv", index = False)

In [42]:
save

'efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0'

In [43]:
model_file

'./efficientnet-b7_600_freeze_gridmask_augmix_fulltrain_best_fold0.pth'